In [12]:
# Day 9 — Bias Mitigation (with robust DIR fallback)
# Paste into a notebook cell and run.
# Requirements: aif360, scikit-learn, pandas, numpy, matplotlib, seaborn
# If you can install packages, run:
#   pip install aif360 BlackBoxAuditing seaborn
# Then run this cell.

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import ClassificationMetric

os.makedirs("reports", exist_ok=True)
sns.set(style="whitegrid")

# -----------------------------
# 1) Load dataset (robust)
# -----------------------------
# tries common paths; will create small dummy dataset if none found
candidates = ["data/cleaned_resume.csv", "notebooks/data/cleaned_resume.csv", "cleaned_resume.csv"]
csv_path = None
for p in candidates:
    if os.path.exists(p):
        csv_path = p
        break

if csv_path is None:
    for root, _, files in os.walk("."):
        for f in files:
            if f.lower().endswith(".csv") and "cleaned" in f.lower() and "resume" in f.lower():
                csv_path = os.path.join(root, f)
                break
        if csv_path:
            break

if csv_path is None:
    print("No cleaned_resume.csv found — creating small dummy dataset at data/cleaned_resume.csv")
    df = pd.DataFrame({
        "resume_text": [
            "Experienced software engineer skilled in Python and ML",
            "Frontend developer with React and JavaScript expertise",
            "Data analyst with SQL and R experience",
            "Marketing specialist with SEO background",
            "AI researcher with NLP focus",
            "Junior developer with Python experience",
            "Senior data scientist with deep learning experience",
            "Product manager with strong communication skills"
        ],
        "gender": ["male", "female", "male", "female", "male", "female", "male", "female"],
        "label": [1,1,0,0,1,0,1,0]
    })
    csv_path = "data/cleaned_resume.csv"
    df.to_csv(csv_path, index=False)
else:
    df = pd.read_csv(csv_path)

print("Using dataset:", csv_path)
print(df.head())

# -----------------------------
# 2) Sanity: required columns
# -----------------------------
for col in ["resume_text","gender","label"]:
    if col not in df.columns:
        raise ValueError(f"Missing required column: {col}")

# Map gender to numeric (male=1 privileged, female=0 unprivileged)
df['gender_mapped'] = df['gender'].map(lambda x: 1 if str(x).strip().lower()=="male" else 0).astype(int)

# -----------------------------
# 3) Text -> numeric via TF-IDF
# -----------------------------
tfidf = TfidfVectorizer(max_features=200, ngram_range=(1,2))
X_text = tfidf.fit_transform(df['resume_text'].fillna("")).toarray()
tf_cols = [f"tfidf_{i}" for i in range(X_text.shape[1])]
df_tf = pd.DataFrame(X_text, columns=tf_cols, index=df.index)

df_num = pd.concat([df_tf, df[['gender_mapped','label']]], axis=1)
df_num = df_num.rename(columns={"gender_mapped":"gender"})  # AIF360 expects the protected attr name in df

# -----------------------------
# 4) Create AIF360 StandardDataset
# -----------------------------
aif_ds = StandardDataset(
    df_num,
    label_name='label',
    favorable_classes=[1],
    protected_attribute_names=['gender'],
    privileged_classes=[[1]]
)
print("AIF360 dataset created. features shape:", aif_ds.features.shape)

# -----------------------------
# 5) Train/test split (AIF360)
# -----------------------------
train, test = aif_ds.split([0.7], shuffle=True)
X_train, y_train = train.features, train.labels.ravel()
X_test, y_test = test.features, test.labels.ravel()

# -----------------------------
# 6) Helper: compute metrics via AIF360 ClassificationMetric
# -----------------------------
def compute_metrics(aif_test, y_pred, method):
    aif_pred = aif_test.copy()
    aif_pred.labels = y_pred.reshape(-1,1)
    metric = ClassificationMetric(aif_test, aif_pred,
                                  unprivileged_groups=[{'gender': 0}],
                                  privileged_groups=[{'gender': 1}])
    acc = accuracy_score(aif_test.labels.ravel(), y_pred)
    f1 = f1_score(aif_test.labels.ravel(), y_pred, zero_division=0)
    spd = metric.statistical_parity_difference()
    eod = metric.equal_opportunity_difference()
    return {'Method': method, 'Accuracy': acc, 'F1': f1, 'SPD': spd, 'EOD': eod}

# -----------------------------
# 7) Baseline
# -----------------------------
clf_base = LogisticRegression(max_iter=1000)
clf_base.fit(X_train, y_train)
y_base = clf_base.predict(X_test)
base_metrics = compute_metrics(test, y_base, "Baseline")
print("Baseline:", base_metrics)

# -----------------------------
# 8) Reweighing (pre-processing)
# -----------------------------
rw = Reweighing(unprivileged_groups=[{'gender':0}], privileged_groups=[{'gender':1}])
rw.fit(train)
train_rw = rw.transform(train)  # includes instance_weights

clf_rw = LogisticRegression(max_iter=1000)
clf_rw.fit(train_rw.features, train_rw.labels.ravel(), sample_weight=train_rw.instance_weights)
y_rw = clf_rw.predict(X_test)
rw_metrics = compute_metrics(test, y_rw, "Reweighing")
print("Reweighing:", rw_metrics)

# -----------------------------
# 9) Try to use official DIR; fallback if BlackBoxAuditing not installed
# -----------------------------
use_official_dir = True
try:
    # attempt to import and construct the official DIR (this import happens inside AIF360 object)
    from aif360.algorithms.preprocessing import DisparateImpactRemover
    # create instance to check dependency
    _ = DisparateImpactRemover(repair_level=1.0)
    print("Official DisparateImpactRemover available.")
except Exception as e:
    print("Official DisparateImpactRemover NOT available (missing dependency). Falling back to residualization repair.")
    # print(e)
    use_official_dir = False

if use_official_dir:
    # Official path
    DIR = DisparateImpactRemover(repair_level=1.0)
    train_dir = DIR.fit_transform(train)
    test_dir = DIR.transform(test)
    clf_dir = LogisticRegression(max_iter=1000)
    clf_dir.fit(train_dir.features, train_dir.labels.ravel())
    y_dir = clf_dir.predict(test_dir.features)
    dir_metrics = compute_metrics(test_dir, y_dir, "DisparateImpactRemover")
    print("DIR (official) metrics:", dir_metrics)
else:
    # Fallback: residualize each numeric feature to remove linear dependence on protected attribute
    # Train residualizers on TRAIN features using TRAIN protected attribute
    print("Running residualization fallback (linear removal of protected attribute signal).")
    # prepare numpy arrays
    Xtr = train.features  # shape (n_train, n_features)
    Xte = test.features
    prot_tr = train.protected_attributes[:, 0].reshape(-1,1)  # gender 0/1
    # Fit linear regression per feature: X_feature ~ prot_tr and compute residuals
    lr = LinearRegression()
    Xtr_res = np.zeros_like(Xtr)
    Xte_res = np.zeros_like(Xte)
    for j in range(Xtr.shape[1]):
        # fit model feature_j ~ prot_tr
        lr.fit(prot_tr, Xtr[:, j])
        pred_tr = lr.predict(prot_tr)
        res_tr = Xtr[:, j] - pred_tr
        Xtr_res[:, j] = res_tr
        # apply same model to test prot attribute
        pred_te = lr.predict(test.protected_attributes[:,0].reshape(-1,1))
        res_te = Xte[:, j] - pred_te
        Xte_res[:, j] = res_te
    # Now train classifier on residualized features
    clf_dir = LogisticRegression(max_iter=1000)
    clf_dir.fit(Xtr_res, train.labels.ravel())
    y_dir = clf_dir.predict(Xte_res)
    # For metric computation, we need a dataset object for test_dir-like shape
    # We'll create copies of test and replace features with Xte_res
    test_dir = test.copy()
    test_dir.features = Xte_res
    dir_metrics = compute_metrics(test_dir, y_dir, "DIR_fallback_residualize")
    print("DIR fallback metrics:", dir_metrics)

# -----------------------------
# 10) Collect and save results
# -----------------------------
results = pd.DataFrame([base_metrics, rw_metrics, dir_metrics])[['Method','Accuracy','F1','SPD','EOD']]
results.to_csv("reports/day9_bias_mitigation_results.csv", index=False)
print("\nSaved results to reports/day9_bias_mitigation_results.csv")
print(results)

# -----------------------------
# 11) Plots and tradeoff charts
# -----------------------------
# Performance & fairness bar charts
plt.figure(figsize=(8,5))
sns.barplot(data=results.melt(id_vars='Method', value_vars=['Accuracy','F1']),
            x='Method', y='value', hue='variable')
plt.title("Performance: Accuracy & F1")
plt.savefig("reports/day9_performance.png", dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(8,5))
sns.barplot(data=results.melt(id_vars='Method', value_vars=['SPD','EOD']),
            x='Method', y='value', hue='variable')
plt.title("Fairness: SPD & EOD")
plt.savefig("reports/day9_fairness.png", dpi=300, bbox_inches='tight')
plt.close()

# Trade-off plots
plt.figure(figsize=(7,6))
sns.scatterplot(data=results, x='Accuracy', y='SPD', hue='Method', s=140)
for _, r in results.iterrows():
    plt.text(r['Accuracy']+1e-4, r['SPD'], r['Method'])
plt.axhline(0, linestyle='--', color='gray')
plt.title("Accuracy vs SPD")
plt.savefig("reports/day9_tradeoff_accuracy_spd.png", dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(7,6))
sns.scatterplot(data=results, x='Accuracy', y='EOD', hue='Method', s=140)
for _, r in results.iterrows():
    plt.text(r['Accuracy']+1e-4, r['EOD'], r['Method'])
plt.axhline(0, linestyle='--', color='gray')
plt.title("Accuracy vs EOD")
plt.savefig("reports/day9_tradeoff_accuracy_eod.png", dpi=300, bbox_inches='tight')
plt.close()

# Radar chart (normalized)
metrics_plot = results.copy()
metrics_plot['Accuracy_n'] = (metrics_plot['Accuracy'] - metrics_plot['Accuracy'].min()) / (metrics_plot['Accuracy'].max() - metrics_plot['Accuracy'].min() + 1e-9)
metrics_plot['SPD_n'] = 1 - (np.abs(metrics_plot['SPD']) - np.abs(metrics_plot['SPD']).min()) / (np.abs(metrics_plot['SPD']).max() - np.abs(metrics_plot['SPD']).min() + 1e-9)
metrics_plot['EOD_n'] = 1 - (np.abs(metrics_plot['EOD']) - np.abs(metrics_plot['EOD']).min()) / (np.abs(metrics_plot['EOD']).max() - np.abs(metrics_plot['EOD']).min() + 1e-9)

plot_cols = ['Accuracy_n','SPD_n','EOD_n']
labels = ['Accuracy','Fairness SPD (higher→better)','Fairness EOD (higher→better)']

angles = np.linspace(0,2*np.pi,len(plot_cols),endpoint=False).tolist()
angles += angles[:1]

plt.figure(figsize=(8,8))
ax = plt.subplot(111, polar=True)
for _, row in metrics_plot.iterrows():
    vals = row[plot_cols].tolist()
    vals += vals[:1]
    ax.plot(angles, vals, linewidth=2, label=row['Method'])
    ax.fill(angles, vals, alpha=0.15)
ax.set_thetagrids(np.degrees(angles[:-1]), labels)
ax.set_ylim(0,1)
plt.title("Radar: Accuracy vs Fairness (normalized)")
plt.legend(loc='upper right', bbox_to_anchor=(1.3,1.1))
plt.savefig("reports/day9_radar.png", dpi=300, bbox_inches='tight')
plt.close()

print("✅ Plots saved to reports/:")
print(" - day9_performance.png")
print(" - day9_fairness.png")
print(" - day9_tradeoff_accuracy_spd.png")
print(" - day9_tradeoff_accuracy_eod.png")
print(" - day9_radar.png")

print("\n✅ Day 9 completed. If you want, I can now:")
print("  • help interpret the metrics (short write-up),")
print("  • add post-processing mitigators (Day 10),")
print("  • or prepare a polished PDF/slide with the reports/ images.")


Using dataset: data/cleaned_resume.csv
                                         resume_text  gender  label
0  Experienced software engineer skilled in Pytho...    male      1
1  Frontend developer with React and JavaScript e...  female      1
2             Data analyst with SQL and R experience    male      0
3           Marketing specialist with SEO background  female      0
4                       AI researcher with NLP focus    male      1
AIF360 dataset created. features shape: (5, 53)
Baseline: {'Method': 'Baseline', 'Accuracy': 0.5, 'F1': 0.6666666666666666, 'SPD': np.float64(nan), 'EOD': np.float64(nan)}
Reweighing: {'Method': 'Reweighing', 'Accuracy': 0.5, 'F1': 0.6666666666666666, 'SPD': np.float64(nan), 'EOD': np.float64(nan)}
Official DisparateImpactRemover NOT available (missing dependency). Falling back to residualization repair.
Running residualization fallback (linear removal of protected attribute signal).
DIR fallback metrics: {'Method': 'DIR_fallback_residualize', 'Ac

/Users/chiragrgowda/Documents/resume-fairness/venv/lib/python3.13/site-packages/aif360/metrics/classification_metric.py:673: RuntimeWarning: invalid value encountered in scalar divide
  return (self.num_pred_positives(privileged=privileged)
/Users/chiragrgowda/Documents/resume-fairness/venv/lib/python3.13/site-packages/aif360/metrics/classification_metric.py:278: RuntimeWarning: invalid value encountered in scalar divide
  TPR=TP / P, TNR=TN / N, FPR=FP / N, FNR=FN / P,
/Users/chiragrgowda/Documents/resume-fairness/venv/lib/python3.13/site-packages/aif360/metrics/classification_metric.py:279: RuntimeWarning: invalid value encountered in scalar divide
  GTPR=GTP / P, GTNR=GTN / N, GFPR=GFP / N, GFNR=GFN / P,
/Users/chiragrgowda/Documents/resume-fairness/venv/lib/python3.13/site-packages/aif360/algorithms/preprocessing/reweighing.py:67: RuntimeWarning: divide by zero encountered in scalar divide
  self.w_p_unfav = n_unfav*n_p / (n*n_p_unfav)
/Users/chiragrgowda/Documents/resume-fairness/

✅ Plots saved to reports/:
 - day9_performance.png
 - day9_fairness.png
 - day9_tradeoff_accuracy_spd.png
 - day9_tradeoff_accuracy_eod.png
 - day9_radar.png

✅ Day 9 completed. If you want, I can now:
  • help interpret the metrics (short write-up),
  • add post-processing mitigators (Day 10),
  • or prepare a polished PDF/slide with the reports/ images.
